### Overview

This is a demo to introduce the step-by-step analysis of LongcellPre, which aims for users who want to use intermediate results from LongcellPre or understand the details about this tool.

For users who want to directly apply this tool to the data to do preprocessing, you can simply run:

In [ ]:
library(future)
library(Longcellsrc)
library(LongcellPre)

# update your input path here
fastq = "path of your input fastq or fq.gz"
barcodes = "path of your input cell barcode whitelist"
gtf_path = "path of your gtf annotation"
genome_path = "path of your genome annotation"
minimap_bed_path = "path of your bed annotation for minimap2, can be generated from gtf" //unnecessary
genome_name = "the genome name used for mapping, ex. hg38"
toolkit = your 10X sequencing toolkit
work_dir = "The output directory"

# specify the path for those tools
samtools = "samtools"
minimap2 = "minimap2"
bedtools = "bedtools"

RunLongcellPre(fastq = fastq,barcode_path = barcodes,toolkit = toolkit,
               genome_path = genome_path,genome_name = genome_name,
               gtf_path = gtf_path,minimap_bed_path = minimap_bed_path,work_dir = work_dir,
               samtools = samtools, minimap2 = minimap2,bedtools = bedtools,cores = 4, strategy="multicore")

### Step-by-step demo for LongcellPre

We provide a demo data with 200 cells and 3 genes. This data is a subset of the colorectal metastasis sample we used in the paper. The data and corresponding annotations can be downloaded from: 
https://www.dropbox.com/scl/fo/21tw8rrkaancani0fzq3t/AKNHUk06onR2c2dYuB4wXWY?rlkey=1zikug28qr9ziw2cdsgelrm9p&st=ypm9m00i&dl=0

#### load necessary libraries and initiation

In [1]:
library(future)
library(Longcellsrc)
library(LongcellPre)

In [4]:
work_dir = "./demo_out"
init(work_dir)

#### build annotation

The annotation used by LongcellPre can be generated from the isoform annotation in gtf format, for common samples like human and mouse, you can download the gtf reference via gencode: https://www.gencodegenes.org/. Here we would use the subset gtf for our data.

In [2]:
gtf_path = "./demo/gencode.v39.sub.gtf"

In [5]:
refer = annotation(gtf_path = gtf_path, work_dir = work_dir ,overwrite = FALSE)

[1] "Start to build exon annotation based on the gtf file."


Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
Warning message in .get_cds_IDX(mcols0$type, mcols0$phase):
“The "phase" metadata column contains non-NA values for features of type
  stop_codon. This information was ignored.”
OK



The `annotation()` function would generate two tables. The first one record the location for the split non-overlapping exons, and the second one records the location for each annotated exon.

In [6]:
gene_bed = refer[[1]]
exon_gtf = refer[[2]]

In [7]:
head(gene_bed)

,chr,start,end,width,strand,gene,id
,<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>
1,chr8,85214048,85214076,29,-,ENSG00000176731,29
2,chr8,85214077,85214486,410,-,ENSG00000176731,28
3,chr8,85214487,85214501,15,-,ENSG00000176731,27
4,chr8,85214502,85214502,1,-,ENSG00000176731,26
5,chr8,85214503,85214555,53,-,ENSG00000176731,25
6,chr8,85214556,85214631,76,-,ENSG00000176731,24


In [8]:
head(exon_gtf)

start,end,gene,transname,exon_id
<int>,<int>,<chr>,<chr>,<chr>
85214048,85214631,ENSG00000176731,ENST00000619594.5,ENSE00003752010.2
85214077,85214631,ENSG00000176731,ENST00000614462.4,ENSE00003742708.1
85214487,85214631,ENSG00000176731,ENST00000612977.4,ENSE00003718609.1
85214502,85214631,ENSG00000176731,ENST00000612809.4,ENSE00003715535.1
85214502,85214631,ENSG00000176731,ENST00000545322.5,ENSE00003715535.1
85214502,85214631,ENSG00000176731,ENST00000615071.1,ENSE00003750870.1


The first table would be used to guide searching the reads for each gene in the bam file, which is necessary. The second one is used as the canonical isoform annotation and is only necessary when you want to map your reads to the canonical isoforms to get the cell by isoform matrix. If this table is not provided, isoform can also be extracted but stored as a string of splicing sites.

When there is no gtf file as the isoform anotation for the sample,the `annotation()` function can accept a gene bed file to indicate the location of targeted genes. LongcellPre will search reads for targeted genes based on the bed.

#### cell barcode match and reads extraction

This part can be achieve by the function `reads_extract_bc()`. This function contains three main steps:

(1) Trim the adapter sequence and identify the cell barcode and UMI for each read.

(2) Map the trimmed fastq file to the reference genome to get the bam file. Common reference genome can be downloaded from https://www.10xgenomics.com/cn/support/software/cell-ranger/downloads
Here we would use the subset genome for convenience.

(3) Extract the isoform information for each read. The read extraction would be guided from the `gene_bed` annotation output from `annotation`. The necessary information in the `gene_bed` here is the location of the gene and also its strand information. If you are only interested in serveral target genes, you could filter out other genes in the `gene_bed`, then reads aligned to other genes won't be searched.

In [10]:
fastq = "./demo/example.fq.gz"
barcodes = "./demo/barcodes.txt"
genome_path = "./demo/genome.fa"

# Bed annotation generated from gtf can help minimap2 map reads at splicing junctions,
# this bed can be accepted by the "minimap_bed_path" parameter. In this demo we ignore
# this input.
# minimap_bed_path = "The path of bed annotation generated from gtf "
genome_name = "hg38"
protocol = "10X"
toolkit = 5

# specify the path for those tools
samtools = "samtools"
minimap2 = "minimap2"
bedtools = "bedtools"

In [35]:
plan(strategy = "multicore",workers = 4)
reads_bc = reads_extract_bc(fastq_path = fastq,barcode_path = barcodes,gene_bed= gene_bed,
                           genome_path = genome_path,genome_name = genome_name,
                           toolkit = toolkit,protocol = protocol, work_dir = work_dir,
                           minimap2 = minimap2, samtools = samtools, bedtools = bedtools)
bc = reads_bc[[1]]
qual = reads_bc[[2]]

Start to do barcode match:In total 162206 reads out of 196661 reads are identified with a valid adapter.
Within them, 161921 reads are identified with a valid tag region.
121120  out of  161921 reads are identified with a vaild cell barcode.
After filtering,  120333  reads with barcodes are preserved, with mean edit distance as  0.4166771 .
Barcode match took 2.42 mins
 
  Elapsed_Time_sec Total_RAM_Used_MiB Peak_RAM_Used_MiB
1          138.866                9.7             103.5
Start to map polished fastq to genome:[1] "The command for mapping is:"
[1] "minimap2 -ax splice -uf --sam-hit-only -t 1 ./demo/genome.fa ./demo_out/polish.fq.gz | samtools view -bS -@ 1 - | samtools sort - -@ 1 -o ./demo_out/bam/polish.bam && samtools index ./demo_out/bam/polish.bam"
Start to extract isoforms:

Warning message in inner_join(bc, reads, by = c(name = "qname")):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 6035 of `x` matches multiple rows in `y`.
ℹ Row 62630 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship = "many-to-many"` to silence this warning.”


Isoform extraction took 41.28 secs
 
  Elapsed_Time_sec Total_RAM_Used_MiB Peak_RAM_Used_MiB
1            37.62               15.1              85.7


This function will return two dataframes, the first one record the cell barcode, UMI, isoform and polyA existence information for each read, while the second one records the distribution of Needleman score between the adapter aside the confidently identified cell barcode (edit distance = 0) and its original sequence. The second table is used to evaluate the data quality and be the guidance to help filter scattered UMI clusters in the UMI deduplication step.

Those returns would also be saved into files along with the bam file in the output directory.

#### UMI deduplication and isoform correction

This part can be achieve by the function `umi_count_corres()`. This function contains three main steps:

(1) Cluster reads with the same or similar UMI for each cell into group.

(2) Correct the wrong mapping and trunction for each UMI cluster to get polished isoform representation

(3) If given the canonical isoform annotation (the `exon_gtf` from the annotation step),this function would try to map each read to a canonical isoform and build a cell-by-isoform matrix.

In [34]:
plan(strategy = "multicore",workers = 4)
umi_count_corres(data = bc,qual = qual,file.path(work_dir,"out"),
                 gene_bed = gene_bed,gtf = exon_gtf)

Start to do UMI deduplication:UMI deduplication took 3.22 mins
 
  Elapsed_Time_sec Total_RAM_Used_MiB Peak_RAM_Used_MiB
1          189.633                0.3              18.3
Start to do isoform alignment:Isoform alignment took 2.11 mins
 
  Elapsed_Time_sec Total_RAM_Used_MiB Peak_RAM_Used_MiB
1          123.868                0.1              28.5


[1] 0

This function has no returns but it would output the single cell isoform quantification into the output folder.